In [ ]:
# Import necessary libraries
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Conv2DTranspose, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Accuracy, Precision, Recall
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load and preprocess data
def load_data(data_path):
    images = []
    masks = []
    for filename in os.listdir("C:\Users\Rajvansh\OneDrive\Documents\CODEBUGGED\Listado de fotos existentes a limpiar.xlsx")
        if filename.endswith(".jpg"):
            img_path = os.path.join("C:\Users\Rajvansh\OneDrive\Documents\CODEBUGGED\Listado de fotos existentes a limpiar.xlsx", filename)
            mask_path = os.path.join("C:\Users\Rajvansh\OneDrive\Documents\CODEBUGGED\Listado de fotos existentes a limpiar.xlsx", filename.split('.')[0] + '_mask.jpg')
            image = cv2.imread(img_path)
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            images.append(image)
            masks.append(mask)
    return np.array(images), np.array(masks)

train_images, train_masks = load_data(""C:\Users\Rajvansh\OneDrive\Documents\CODEBUGGED\Listado de fotos existentes a limpiar.xlsx"")
test_images, test_masks = load_data(""C:\Users\Rajvansh\OneDrive\Documents\CODEBUGGED\Medidas modelos TELKO 2024.xlsx"")

# Normalize images and masks
train_images = train_images / 255.0
train_masks = train_masks / 255.0
test_images = test_images / 255.0
test_masks = test_masks / 255.0

# Define U-Net model
def unet(input_size=(256, 256, 3)):
    inputs = Input(input_size)
    
    # Encoder
    conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    # Decoder
    conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv3)
    up4 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv3)
    up4 = concatenate([conv2, up4], axis=3)
    conv4 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(up4)
    conv4 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv4)
    
    up5 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv4)
    up5 = concatenate([conv1, up5], axis=3)
    conv5 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(up5)
    conv5 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv5)
    
    # Output layer
    outputs = Conv2D(1, 1, activation='sigmoid')(conv5)

    model = Model(inputs=inputs, outputs=outputs)
    return model

# Define IOU metric
def mean_iou(y_true, y_pred):
    intersection = np.logical_and(y_true, y_pred)
    union = np.logical_or(y_true, y_pred)
    iou_score = np.sum(intersection) / np.sum(union)
    return iou_score

# Compile the model
model = unet()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', Precision(), Recall(), mean_iou])

# Data augmentation
datagen = ImageDataGenerator(rotation_range=20, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.1, zoom_range=0.1, horizontal_flip=True, fill_mode='nearest')

# Split train and validation data
X_train, X_val, y_train, y_val = train_test_split(train_images, train_masks, test_size=0.1, random_state=42)

# Train the model
history = model.fit(datagen.flow(X_train, y_train, batch_size=16), epochs=50, validation_data=(X_val, y_val))

# Evaluate the model
scores = model.evaluate(test_images, test_masks, verbose=1)
print("Test Loss:", scores[0])
print("Test Accuracy:", scores[1])
print("Test Precision:", scores[2])
print("Test Recall:", scores[3])
print("Test Mean IoU:", scores[4])

# Perform inference and change background to white
def segment_glasses(image_path, model):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (256, 256))
    image = image / 255.0
    image = np.expand_dims(image, axis=0)
    mask = model.predict(image)
    mask = (mask > 0.5).astype(np.uint8)
    mask = np.squeeze(mask, axis=0)
    mask = cv2.resize(mask, (image.shape[2], image.shape[1]))
    
    # Change background to white
    background = np.ones_like(image) * 255
    segmented_image = np.where(mask > 0, image, background)
    
    return segmented_image

# Example inference
example_image_path = "test/example.jpg"
segmented_image = segment_glasses(example_image_path, model)

# Display original and segmented image
original_image = cv2.imread(example_image_path)
cv2.imshow("Original Image", original_image)
cv2.imshow("Segmented Image", segmented_image[0])
cv2.waitKey(0)
cv2.destroyAllWindows()
